In [1]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate, HumanMessagePromptTemplate
from langchain.document_loaders import PyPDFLoader, TextLoader
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI
import pandas as pd
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from pydantic import BaseModel
from langchain.chat_models import ChatOpenAI 
from langchain.text_splitter import SpacyTextSplitter
from dotenv import load_dotenv
import pdb
#import tiktoken


In [2]:
load_dotenv()

True

In [3]:
llm = ChatOpenAI(model="gpt-3.5-turbo",temperature=0)
embeddings = OpenAIEmbeddings(deployment="embeddings model", chunk_size=1) 

In [9]:
def load_pdf(pdf):
    ''' This utility is used to convert the pdfs into chunks of texts '''
    #loader = PyPDFLoader(pdf)
    loader = TextLoader(pdf,encoding='MacRoman')
    documents = loader.load()

    text_splitter = CharacterTextSplitter(chunk_size = 900, chunk_overlap = 200)
    texts = text_splitter.split_documents(documents)
    return texts
    

In [10]:
pdf_text = load_pdf("../files/(1) Master Paper - Ultrafast quantum random number generation using quantum phase fluctuations.pdf")
doc_search_pdf = FAISS.from_documents(pdf_text, embeddings)
doc_search_pdf.save_local("./fiass_db/"+"Quantum Computing.pdf")

Created a chunk of size 2179, which is longer than the specified 900
Created a chunk of size 17770, which is longer than the specified 900
Created a chunk of size 6607, which is longer than the specified 900
Created a chunk of size 3418, which is longer than the specified 900
Created a chunk of size 36950, which is longer than the specified 900
Created a chunk of size 1393, which is longer than the specified 900
Created a chunk of size 13512, which is longer than the specified 900
Created a chunk of size 70175, which is longer than the specified 900
Created a chunk of size 12164, which is longer than the specified 900
Created a chunk of size 15614, which is longer than the specified 900
Created a chunk of size 1792, which is longer than the specified 900
Created a chunk of size 1077, which is longer than the specified 900
Created a chunk of size 1023, which is longer than the specified 900
Created a chunk of size 1556, which is longer than the specified 900
Created a chunk of size 1632

ImportError: Could not import tiktoken python package. This is needed in order to for OpenAIEmbeddings. Please install it with `pip install tiktoken`.

In [ ]:
persisted_vectorstore = FAISS.load_local("./fiass_db/"+"Quantum Computing.pdf", embeddings)

    # Use RetrievalQA chain for orchestration
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=persisted_vectorstore.as_retriever())
result = qa.run("What is Quantum Computing")
print(result)

ImportError: Could not import faiss python package. Please install it with `pip install faiss` or `pip install faiss-cpu` (depending on Python version).

In [ ]:
def query_pdf(query):
    # Load document using PyPDFLoader document loader
    loader = PyPDFLoader("Downloads/Quantum Computing.pdf")
    documents = loader.load()
    # Split document in chunks
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=30, separator="\n")
    docs = text_splitter.split_documents(documents=documents)

    embeddings = OpenAIEmbeddings()
    # Create vectors
    vectorstore = FAISS.from_documents(docs, embeddings)
    # Persist the vectors locally on disk
    vectorstore.save_local("faiss_index_constitution")

    # Load from local storage
    persisted_vectorstore = FAISS.load_local("faiss_index_constitution", embeddings)

    # Use RetrievalQA chain for orchestration
    qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=persisted_vectorstore.as_retriever())
    result = qa.run(query)
    print(result)



In [ ]:

def main():
    query = input("Type in your query: \n")
    while query != "exit":
        query_pdf(query)
        query = input("Type in your query: \n")

In [ ]:
main()

ImportError: pypdf package not found, please install it with `pip install pypdf`